In [1]:
using LinearAlgebra

## Outros meios de resolver sistemas

Vamos testar alguns métodos de resolução de sistemas e analisar o tempo gasto por cada um deles.

Primeiramente, vejamos a resolução de um sistema aleatório usando cálculo da inversa:

In [2]:
function t1(n, k)
   for i = 1:k
       A = rand(n, n)
       b = rand(n)
        inv(A) * b
   end
end

t1 (generic function with 1 method)

In [3]:
@time t1(300, 200)

  0.973613 seconds (2.01 k allocations: 305.506 MiB, 2.89% gc time)


Podemos também analisar a fatoração $LU$ da máquina.

In [4]:
function t2(n, k)
   for i = 1:k
       A = rand(n, n)
       b = rand(n)
        lu(A) \ b
   end
end

t2 (generic function with 1 method)

In [5]:
@time t2(300, 200)

  0.504163 seconds (1.60 k allocations: 276.160 MiB, 4.34% gc time)


# Fatoração LU e resolução de sistemas

Para resolver um sistema da forma $Ax = b,$ podemos encontrar $L$ e $U,$ tais que $$A=LU.$$ Então, operamos $$Ly=b \text{    e   } Ux=y.$$

## Fatoração LU

Neste problema, dada uma matriz $A,$ queremos ver a sua fatoração $LU.$

In [6]:
show_LU(A)=
begin
    n, n = size(A)
    #criamos a matriz L
    L = zeros(n, n)
    L[1, 1] = 1
    #criamos a matriz U
    U = copy(A)
    
    for j = 1 : n - 1
        for i = j + 1 : n
            #fazemos a eliminação gaussiana
            t = U[i, j] / U[j, j]
            U[i, :] = U[i, :] - t * U[j, :]
            #a diagonal principal de L é preenchida por 1
            L[i, i] = 1
            #guardamos t(pivô) na matriz L
            L[i, j] = t
        end
    end
    display(L)
    display(U)
end

show_LU (generic function with 1 method)

In [7]:
A = [6.0 18 22; 4 7 7; 2 2 2]

3×3 Array{Float64,2}:
 6.0  18.0  22.0
 4.0   7.0   7.0
 2.0   2.0   2.0

In [8]:
show_LU(A)

3×3 Array{Float64,2}:
 1.0       0.0  0.0
 0.666667  1.0  0.0
 0.333333  0.8  1.0

3×3 Array{Float64,2}:
 6.0  18.0  22.0
 0.0  -5.0  -7.66667
 0.0   0.0   0.8

Note que no programa anterior estamos ocupando mais espaço na memória do que o necessário, uma vez que podemos guardar $L$ e $U$ em uma única matriz, ou ainda, na matriz inicial $A.$ Este segundo meio não vamos trabahar aqui para preservarmos a matriz $A.$ Vejamos a seguir.

In [9]:
calculate_LU(A)=
begin
    n, n = size(A)
    #criamos a matriz LU que irá guardar L na parte inferior e U na parte superior
    LU = copy(A)
    
    for j = 1 : n - 1
        for i = j + 1 : n
            t = LU[i, j] / LU[j, j]
            #armazenamos a U na diagonal principal e posições acima
            LU[i, j : n] = LU[i, j : n] - t * LU[j, j : n]
            #armazenamos L nas entradas abaixo da diagonal principal
            LU[i, j] = t
        end
    end
    
    return LU
end

calculate_LU (generic function with 1 method)

In [10]:
calculate_LU(A)

3×3 Array{Float64,2}:
 6.0       18.0  22.0
 0.666667  -5.0  -7.66667
 0.333333   0.8   0.8

### Pivoteamento

Durante o calculo da fatoração $LU$ é necessário realizar algumas divisões para calcular o que chamamos de pivôs. Porém, o que acontece quando o pivô é zero? Vejamos a seguir.

In [11]:
B = [0.0 3 -1; 2 7 23; 1 5 14]
calculate_LU(B)

3×3 Array{Float64,2}:
  0.0    3.0   -1.0
 Inf   -Inf    Inf
 Inf   NaN    NaN

Não é possível calcular a fatoração uma vez que estamos dividindo por zero. Porém, note que se trocassemos as linhas 1 e 2 conseguiriamos calcular a fatoração e não perderíamos informações do sistema. Para isso, vamos usar um processo chamado pivoteamento, onde vamos organizar nossa matriz de forma a evitar pivôs zeros.

A mudança de linhas não afetará o resultado de nosso sistema desde que operemos a mesma alteração de linhas no vetor que contém o lado direito do sistema.

No programa a seguir, vamos calcular a fatoração $LU$ com o pivoteamento, esse programa ira retornar a matriz $LU$ e um vetor $p$ que armazenará a posição final de cada linha da matriz. 

In [12]:
LU_pivoting(A)=
begin
    n,n = size(A)
    LU = copy(A)
    p = zeros(Int, n - 1)
    α = zeros(n)
    
    for j = 1 : n - 1
        
        #realizamos o pivoteamento
        pivo = LU[j,j]
        m = j
        
        for k = j + 1 : n
            if abs(pivo) < abs(LU[k, j])
                pivo = LU[k, j]
                m = k
            end
        end
        
        if abs(pivo) <= 1.0e-8 
            println("Determinante próximo de 0")
        end
        
        #fazemos a troca de linhas quando necessário
        if m != j
            α .= LU[m, :] 
            LU[m, :] = LU[j, :]
            LU[j, :] .= α
            
        end
        
        #guardamos em p
         p[j] = m
        
        #realizamos a fatoração com a matriz com as linhas trocadas
        for i = j + 1 : n
            t = LU[i, j] / LU[j, j]
            @. LU[i, j : n] = LU[i, j : n] - t * LU[j, j : n]
            LU[i, j] = t
        end
    end
   
    return LU, p
end

LU_pivoting (generic function with 1 method)

Agora podemos calcular a fatoração da matriz $B.$ Observemos ainda que o vetor $p$ indica que houve uma troca das linhas 1 e 2.

In [13]:
B = [0.0 3 -1; 2 7 23; 1 5 14]
LU_pivoting(B)

([2.0 7.0 23.0; 0.0 3.0 -1.0; 0.5 0.5 3.0], [2, 2])

## Resolvendo sistemas

Agora que podemos calcular a fatoração $LU$ resta saber como resolver um sistema usando ela.

O programa a seguir ira resolver um sistema da forma $$LUx = b,$$ para isso colocamos a fatoração $LU$ e o vetor com o lado direito do sistema, sem pivoteamento.

In [14]:
solve_LU(LU, b)=
begin
    n, n = size(LU)
    
    #resolve Ly = b
    #note que não é necessário separar a matriz L e a U
    y = zeros(n)
    
    for k = 1 : n
        s = 0 
        for i = 1 : k - 1
            s = s + y[i] * LU[k, i]
        end
        
        y[k] = b[k] - s
    end
    
    #resolve Ux = y
    x = zeros(n)
    
    for k = n : - 1 : 1 
        s = 0
        for i = n : - 1 : k + 1
            s = s + x[i] * LU[k, i]
        end
        
        x[k] = (y[k] - s) / LU[k, k]
    end
    
    return x
end

solve_LU (generic function with 1 method)

Já este próximo programa, colocamos também de entrada o vetor $p$ com o pivoteamento, assim reorganizamos o vetor $b$ com o pivotemento para poder resolver o sistema de forma adequada usando o programa anterior.

In [15]:
solve_pivoting(LU, b, p)=
begin
    n, n = size(LU)  
    α = 0
    
    #organizamos o vetor b de acordo com o pivotemento em p
    for i = 1 : n - 1
        β = b[i]
        b[i] = b[p[i]]
        b[p[i]] = β
    end
    
    b = solve_LU(LU, b)
    return b
end

solve_pivoting (generic function with 1 method)

Por fim, o programa a seguir resolve todo o sistema de forma direta, ou seja, colocamos a matriz original $A$ e o vetor $b$ contendo o lado direito do sistema, o programa então realiza o pivotemaneto, a fatoração e por fim resolve nosso sistema.

In [16]:
solve(A, b)=
begin
    A, p = LU_pivoting(A) 
    b = solve_pivoting(A, b, p)
    return b
end

solve (generic function with 1 method)

In [17]:
A2 = [1.0 2 -3 4; 4 8 12 -8; 2 3 2 1; -3 -1 1 -4]
b2 = [3.0; 60; 1; 5]
solve(A2, b2)

4-element Array{Float64,1}:
  12.0
   6.0
 -13.0
 -15.0

In [18]:
A3 = [732.0 8 1 0 46; 54 14 16 21 2; 7 7 27 37 47; 14 21 8 64 123; 28 17 15 14 13]
b3 = [28.0; 1; 0; -30; 7]
solve(A3, b3)

5-element Array{Float64,1}:
  0.02834897729427483
  0.16350408268766617
  0.9314243212809877
 -0.8543084602362935
  0.10889329647409797